In [2]:
import tensorflow as tf
import coremltools as ct

from datetime import datetime, timedelta
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

print(tf.__version__)

2.3.1


In [3]:
def normalize(x):
    x = 2 * ( (x - np.min(x)) / (np.max(x) - np.min(x)) ) - 1
    return x


def demean(x):
    x = x - np.mean(x)
    return x

In [4]:
clss = [0,1,2,4] # mask_off, normal_breathing, talking, cough
X_train, Y_train, X_test, Y_test = [], [], [], []


data_files = []
for root, subdirs, files in os.walk('data/train'):
    if len(files) == 0: continue
    
    for file in files:
        data_files.append(f'{root.split("/")[-1]}/{file}')

for file in data_files:
    train = np.load(f'data/train/{file}')
    test = np.load(f'data/test/{file}')

    # normalize sample
    for s in train:
        sample = np.vstack((
                demean(s[0]),
                demean(s[1])
            ))
        X_train.append(sample)
        if 'mask_off' in file: Y_train.append(0)
        elif 'normal_breathing' in file: Y_train.append(1)
        elif 'talking' in file: Y_train.append(2)
        elif 'cough' in file: Y_train.append(3)

    for s in test:
        X_test.append(
            np.vstack((
                demean(s[0]),
                demean(s[1])
            ))
        )
        if 'mask_off' in file: Y_test.append(0)
        elif 'normal_breathing' in file: Y_test.append(1)
        elif 'talking' in file: Y_test.append(2)
        elif 'cough' in file: Y_test.append(3)
    
X_train, Y_train, X_test, Y_test = np.array(X_train, dtype='float64'), np.array(Y_train, dtype='float64'), np.array(X_test, dtype='float64'), np.array(Y_test, dtype='float64')

In [5]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(2,51), name='input'),
    tf.keras.layers.Dense(16, activation='relu'),
    # tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(4, name='classification')
])
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True ),
    metrics=['accuracy']
)

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (Flatten)              (None, 102)               0         
_________________________________________________________________
dense (Dense)                (None, 16)                1648      
_________________________________________________________________
classification (Dense)       (None, 4)                 68        
Total params: 1,716
Trainable params: 1,716
Non-trainable params: 0
_________________________________________________________________


In [10]:
X_test.shape

(771, 2, 51)

In [58]:
model.fit(np.nan_to_num(X_train), Y_train, epochs=1000, batch_size=len(X_train))

1/1 [==============================] - 0s 840us/step - loss: 0.2092 - accuracy: 0.9505
Epoch 804/1000
1/1 [==============================] - 0s 851us/step - loss: 0.2089 - accuracy: 0.9505
Epoch 805/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.2086 - accuracy: 0.9505
Epoch 806/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.2083 - accuracy: 0.9505
Epoch 807/1000
1/1 [==============================] - 0s 877us/step - loss: 0.2080 - accuracy: 0.9505
Epoch 808/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.2077 - accuracy: 0.9505
Epoch 809/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.2074 - accuracy: 0.9505
Epoch 810/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.2072 - accuracy: 0.9505
Epoch 811/1000
1/1 [==============================] - 0s 643us/step - loss: 0.2069 - accuracy: 0.9505
Epoch 812/1000
1/1 [==============================] - 0s 748us/step - loss: 0.2066 - accuracy: 0.9510
Epoch

In [59]:
test_loss, test_acc = model.evaluate(np.nan_to_num(X_test),  Y_test, verbose=2)
print('\nTest accuracy:', test_acc)

25/25 - 0s - loss: 0.3514 - accuracy: 0.9040

Test accuracy: 0.9040207266807556


In [60]:
model.save('simple_nn')
mlmodel = ct.convert('simple_nn') 

Translating MIL ==> MLModel Ops: 100%|██████████| 9/9 [00:00<00:00, 7601.44 ops/s]


In [61]:
mlmodel.author = "Blaine Rothrock"
mlmodel.short_description = "Classification of 3 second temp and pressure data @ 17hz. Classes: Mask Off, Normal Breathing, Talking, Cough. Cough has very little representation"
mlmodel.version = "0.0.1"

In [62]:
mlmodel.save('RespiratoryClassifier.mlmodel')